In [8]:

import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import cv2 
import glob
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator_path = 'WGAN/WGAN_generator_model_new2.pth'#'DCGAN_generator.pth'
model_name = 'WGAN' #'DCGAN'


In [3]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [5]:
import pretrainedmodels
model_name = 'xception'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

In [6]:
class Discriminator2(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator2, self).__init__()
        self.ngpu = ngpu
        self.model = model
        self.model.last_linear = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
        
    def forward(self, input):
        return self.model(input)

In [ ]:
class Discriminator_wgan(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator_wgan, self).__init__()
        self.ngpu = ngpu
        self.model = model
        self.model.last_linear = nn.Sequential(nn.Linear(2048,1)) #, nn.Sigmoid()
        
    def forward(self, input):
        return self.model(input)

In [14]:
batch_size = 32
momentum = 0.9
# lr = 0.001
epochs = 10
log_interval = 10

count1 = 0
count2 = 0

X = []
y = []

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

model_G = Generator(ngpu)
model_G = torch.load(generator_path).to(device)
for i in range(50):
  count1 += 64 
  fixed_noise = torch.randn(64, nz, 1, 1, device=device)
  with torch.no_grad():
    fake = model_G(fixed_noise).detach().cpu()

  X.append(fake)
  y.append(torch.zeros(len(fake)))

X =torch.cat(X, 0)
y = torch.cat(y,0)



cpu
torch.Size([3200, 3, 64, 64]) torch.Size([3200])


In [16]:
count2 = 0
import numpy as np 
from PIL import Image
dataroot = "/vast/aj3281/celeba-dataset"
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
for i in glob.glob(dataroot + '/img_align_celeba/img_align_celeba/*'):
  count2+=1 
  if (count2<=3200):
    im = Image.open(i)
    im = transform(im)
    im = im.unsqueeze(0)
    
    X = torch.cat((X,im ), 0)
    y = torch.cat((y, torch.tensor(1).unsqueeze(0)), 0)
  if(count2>=3200):
    break 

print(count1, count2)

3200 3200


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train), torch.tensor(y_test) 

X_train.shape


my_dataset_train = TensorDataset(X_train, y_train) # create your datset
train_loader = DataLoader(my_dataset_train, batch_size=32, shuffle=True)

my_dataset_test = TensorDataset(X_test, y_test) # create your datset
test_loader = DataLoader(my_dataset_test, batch_size=32, shuffle=True)

<ipython-input-18-310ab84e7a28>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train), torch.tensor(y_test)


In [19]:
model = Discriminator2(ngpu=1).to(device)

In [20]:
lr = 0.0005

batch_size = 64
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = nn.BCELoss()

def train(epoch):
    model.train()
    correct =0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)#.float()
        
        optimizer.zero_grad()
        output = model(data).view(-1)

        loss = loss_func(output, target.float())
        loss.backward()
        optimizer.step()
        pred = torch.zeros(output.data.shape).to(device)
        pred[output.data >=0.5] = 1

        # pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        # if batch_idx % log_interval == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy{:.3f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item(), 100. * correct / len(train_loader.dataset) ))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device).float()
        target = target.to(device).long()
        output = model(data).view(-1)
        validation_loss = loss_func(output, target.float())
        pred = torch.zeros(output.data.shape).to(device)
        pred[output.data >=0.5] = 1
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(test_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    validation()
    model_file = model_name + '_discriminator_model_pretrained' + str(epoch) + '.pth'
    torch.save(model, model_file)
    print('\nSaved model to ' + model_file + '.')

/ext3/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 [5408/5440 (99%)]	Loss: 0.035929	Accuracy97.518

Validation set: Average loss: 0.0000, Accuracy: 959/960 (100%)


Saved model to WGAN_discriminator_model_pretrained1.pth.
Train Epoch: 2 [5408/5440 (99%)]	Loss: 0.001683	Accuracy99.945

Validation set: Average loss: 0.0000, Accuracy: 958/960 (100%)


Saved model to WGAN_discriminator_model_pretrained2.pth.
Train Epoch: 3 [5408/5440 (99%)]	Loss: 0.003541	Accuracy99.926

Validation set: Average loss: 0.0000, Accuracy: 960/960 (100%)


Saved model to WGAN_discriminator_model_pretrained3.pth.
Train Epoch: 4 [5408/5440 (99%)]	Loss: 0.005488	Accuracy99.871

Validation set: Average loss: 0.0000, Accuracy: 959/960 (100%)


Saved model to WGAN_discriminator_model_pretrained4.pth.
Train Epoch: 5 [5408/5440 (99%)]	Loss: 0.000103	Accuracy99.926

Validation set: Average loss: 0.0000, Accuracy: 959/960 (100%)


Saved model to WGAN_discriminator_model_pretrained5.pth.
Train Epoch: 6 [5408/5440 (99%)]	Loss: 0.000021	Accuracy100.000

Valid